# Data Preparation

## Imports

In [1]:
import pandas as pd
import numpy as np
import ast
import pytz
#import remyenv
import matplotlib.pyplot as plt
import xgboost as xgb 
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")


## Read files and concatenate ID orderbooks

In [5]:
# 01.01.2024 00:00 - 31.12.2024 23:00 (CET)
da_prices_df = pd.read_parquet('Data/da_prices_2024.parquet')

In [6]:
# 01.01.2024 01:00 - 01.04.2024 21:00 (UCT)
orderbook_q1 = pd.read_parquet('Data/orderbook_q1.parquet')

# 01.04.2024 00:00 - 01.07.2024 21:00 (UCT)
orderbook_q2 = pd.read_parquet('Data/orderbook_q2.parquet')

# 01.07.2024 00:00 - 01.10.2024 21:00 (UCT)
orderbook_q3 = pd.read_parquet('Data/orderbook_q3.parquet')

# 01.10.2024 00:00 - 01.01.2025 22:00 (UCT)
orderbook_q4 = pd.read_parquet('Data/orderbook_q4.parquet')

In [7]:
orderbook_q2

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask
0,NX_444231,PH-20240401-03,2024-04-01 00:00:00.000 +0000,2024-04-01 01:00:00.000 +0000,10,N_2,NO3,2024-03-31 22:39:07.227 +0000,2024-03-31 22:00:00.000 +0000,"{'X6295172897','X6299316163','X6299434042','X6...","{'X6298407875','X6293723967','X6293723971','X6...","{4300,4148,3611,3601,3600,3521,3521,3521,3521,...","{5269,5300,5300,5300,5400,5500,5500,5500,5600,...","{4000,10000,100,3000,11000,100,200,700,3000,8000}","{200,5000,7000,8000,2000,8000,6000,7000,10000,..."
1,NX_444238,PH-20240401-04,2024-04-01 01:00:00.000 +0000,2024-04-01 02:00:00.000 +0000,10,N_2,NO3,2024-03-31 22:39:23.090 +0000,2024-03-31 22:00:00.000 +0000,"{'X6293919201','X6299264895','X6299315190','X6...","{'X6296305108','X6298927096','X6293723977','X6...","{4200,4200,4094,3739,3739,3722,3711,3711,3656,...","{4963,5240,5300,5300,5300,5300,5400,5400,5400,...","{3000,2000,10000,3500,5000,1800,200,100,100,200}","{5000,400,5000,7000,8000,3000,8000,6000,7000,1..."
2,NX_444245,PH-20240401-05,2024-04-01 02:00:00.000 +0000,2024-04-01 03:00:00.000 +0000,10,N_2,NO3,2024-03-31 22:39:33.786 +0000,2024-03-31 22:00:00.000 +0000,"{'X6299428911','X6299431105','X6299426834','X6...","{'X6299291873','X6299291874','X6293919204','X6...","{4019,4011,4010,4002,4002,4000,3997,3860,3210,...","{4890,4890,4900,5200,5291,5300,5300,5300,5500,...","{5000,5000,1000,3000,100,4200,4300,100,5000,5000}","{5000,5000,6000,5900,300,5000,7000,8000,8000,6..."
3,NX_444252,PH-20240401-06,2024-04-01 03:00:00.000 +0000,2024-04-01 04:00:00.000 +0000,10,N_2,NO3,2024-03-31 22:39:42.530 +0000,2024-03-31 22:00:00.000 +0000,"{'X6299412843','X6299412951','X6299183946','X6...","{'X6299242518','X6299242522','X6299433265','X6...","{3352,3352,3340,3257,2950,2950,2949,2171,1801,...","{3665,3665,4489,4489,4489,4489,4490,4521,4540,...","{2000,4000,3000,100,5000,5000,5000,4000,5000,1...","{5000,3300,5000,4000,5000,4300,500,400,100,200}"
4,NX_444259,PH-20240401-07,2024-04-01 04:00:00.000 +0000,2024-04-01 05:00:00.000 +0000,10,N_2,NO3,2024-03-31 22:39:49.954 +0000,2024-03-31 22:00:00.000 +0000,"{'X6299328810','X6294215040','X6299272048','X6...","{'X6299433898','X6299313658','X6299424351','X6...","{3688,3488,3192,3003,2972,2701,2001,2000,2000,...","{4990,5000,5032,5032,5032,5032,5053,5073,5073,...","{1000,3300,1000,4000,6000,4000,5000,10000,4000...","{4000,1300,100,200,700,3000,8000,5000,7000,5000}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2686315,NX_461925,PH-20240701-20,2024-07-01 17:00:00.000 +0000,2024-07-01 18:00:00.000 +0000,10,N_2,NO3,2024-07-01 09:41:27.133 +0000,2024-06-30 21:59:00.000 +0000,"{'X7440512493','X7440512664','X7440512680','X7...","{'X7440411191','X7440646793','X7437010366','X7...","{3395,3392,3392,3390,3290,3095,3090,2990,2810,...","{3895,3897,5000,5005,5350,5910,9819,9819,9822,...","{5000,5000,2000,5000,10000,5000,10000,10000,25...","{5000,5000,5000,5000,6000,5000,5000,5000,4000,..."
2686316,NX_461932,PH-20240701-21,2024-07-01 18:00:00.000 +0000,2024-07-01 19:00:00.000 +0000,10,N_2,NO3,2024-07-01 08:41:02.203 +0000,2024-06-30 21:59:00.000 +0000,"{'X7440512494','X7440512667','X7440512678','X7...","{'X7440512381','X7440653694','X7436927087','X7...","{3352,3352,3352,3350,3250,3115,3035,3035,3034,...","{3797,3797,5430,5910,13340,13340,13340,13340,1...","{5000,5000,2000,5000,10000,5000,2000,2000,2500...","{5000,2500,5000,5000,5000,5000,4500,10000,5000..."
2686317,NX_461940,PH-20240701-22,2024-07-01 19:00:00.000 +0000,2024-07-01 20:00:00.000 +0000,10,N_2,NO3,2024-07-01 08:41:11.710 +0000,2024-06-30 21:59:00.000 +0000,"{'X7440512495','X7440512682','X7440512357','X7...","{'X7440655056','X7440655061','X7440654916','X7...","{3275,3275,3270,3170,2885,2870,2795,2795,2795,...","{3697,3697,3700,5600,5910,11639,12417,12500,12...","{2500,2000,5000,10000,5000,10000,2500,2000,200...","{2500,5000,10000,4000,5000,1400,5000,2000,400,..

In [8]:
# Concatenate the ID orderbooks
orderbook_df = pd.concat([orderbook_q1, orderbook_q2, orderbook_q3, orderbook_q4])
orderbook_df.to_parquet('Data/orderbook.parquet')

In [10]:
da_prices_df.dtypes

Delivery Start (CET)     object
Delivery End (CET)       object
NO3 Price (EUR)         float64
dtype: object

## Convert columns to datetime

In [9]:
# The coulumns dlvryStart, dlvryEnd and time are in string format. We need to convert them to datetime format.|
orderbook_df['dlvryStart'] = pd.to_datetime(orderbook_df['dlvryStart'])
orderbook_df['dlvryEnd'] = pd.to_datetime(orderbook_df['dlvryEnd'])
orderbook_df['time'] = pd.to_datetime(orderbook_df['time'])

In [10]:
# Convert string columns to datetime objects (without timezone info initially)
da_prices_df['Delivery Start (CET)'] = pd.to_datetime(da_prices_df['Delivery Start (CET)'], format="%d.%m.%Y %H:%M:%S")
da_prices_df['Delivery End (CET)'] = pd.to_datetime(da_prices_df['Delivery End (CET)'], format="%d.%m.%Y %H:%M:%S")

# Localize to CET timezone
cet = pytz.timezone('CET')
da_prices_df['Delivery Start (CET)'] = da_prices_df['Delivery Start (CET)'].dt.tz_localize(cet, ambiguous='infer')
da_prices_df['Delivery End (CET)'] = da_prices_df['Delivery End (CET)'].dt.tz_localize(cet, ambiguous='infer')

# Convert from CET to UCT
da_prices_df['Delivery Start (UTC)'] = da_prices_df['Delivery Start (CET)'].dt.tz_convert('UTC')
da_prices_df['Delivery End (UTC)'] = da_prices_df['Delivery End (CET)'].dt.tz_convert('UTC')

In [13]:
da_prices_df.dtypes

Delivery Start (CET)    datetime64[ns, CET]
Delivery End (CET)      datetime64[ns, CET]
NO3 Price (EUR)                     float64
Delivery Start (UTC)    datetime64[ns, UTC]
Delivery End (UTC)      datetime64[ns, UTC]
dtype: object

In [11]:
# Remove the CET columns
da_prices_df = da_prices_df.drop(columns=['Delivery Start (CET)', 'Delivery End (CET)'])

In [15]:
da_prices_df

,NO3 Price (EUR),Delivery Start (UTC),Delivery End (UTC)
0,29.56,2023-12-31 23:00:00+00:00,2024-01-01 00:00:00+00:00
1,28.46,2024-01-01 00:00:00+00:00,2024-01-01 01:00:00+00:00
2,26.66,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00
3,24.48,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00
4,24.01,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00
...,...,...,...
8779,10.17,2024-12-31 18:00:00+00:00,2024-12-31 19:00:00+00:00
8780,10.64,2024-12-31 19:00:00+00:00,2024-12-31 20:00:00+00:00
8781,7.13,2024-12-31 20:00:00+00:00,2024-12-31 21:00:00+00:00
8782,7.16,2024-12-31 21:00:00+00:00,2024-12-31 22:00:00+00:00


## Merge ID orderbook with DA prices

In [12]:
# Merge the ID orderbook with the DA prices.
df = pd.merge(orderbook_df, da_prices_df, left_on='dlvryStart', right_on='Delivery Start (UTC)', how='left')

In [17]:
df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask,NO3 Price (EUR),Delivery Start (UTC),Delivery End (UTC)
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...","{'X5385371204','X5385374175','X5385353666','X5...","{2247,2100,2000,2000,1751,1443,1212,1205,1205,...","{2350,2378,2409,2497,2514,2559,2590,2600,2657,...","{500,5000,800,1600,1000,700,5000,5200,800,2200}","{10000,2000,5000,1000,200,500,300,10000,4000,500}",26.66,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00
1,NX_426238,PH-20240101-04,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:56.947 +0000,2023-12-31 23:00:00+00:00,"{'X5385372330','X5385372495','X5385371311','X5...","{'X5385372323','X5385372333','X5385373948','X5...","{2061,2061,2051,2000,2000,2000,1920,1467,1000,...","{2251,2251,2262,2299,2300,2433,2447,2448,2551,...","{1900,3000,100,5000,800,1500,400,500,1000,1000}","{200,500,200,3000,10000,500,3600,15000,4000,4000}",24.48,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00
2,NX_426245,PH-20240101-05,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:00.755 +0000,2023-12-31 23:00:00+00:00,"{'X5385359141','X5385367235','X5385367212','X5...","{'X5385364822','X5385364828','X5385364821','X5...","{2000,1950,1927,1917,1916,1320,1319,1319,1319,...","{2127,2127,2129,2150,2200,2400,2400,2401,2401,...","{100,100,1200,9000,2000,4500,1600,500,1000,1000}","{200,5000,400,10000,2000,4000,5000,500,15000,400}",24.01,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00
3,NX_426252,PH-20240101-06,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:06.307 +0000,2023-12-31 23:00:00+00:00,"{'X5385371584','X5385371589','X5385371590','X5...","{'X5385363190','X5385363118','X5383614807','X5...","{1574,1574,1574,1574,1574,1574,1424,1423,1000,...","{1798,1800,1875,2050,2123,2423,2504,2550,2670,...","{1500,5000,1000,2000,1000,5000,2000,5000,1000,...","{10000,10000,10000,5000,15000,1000,1900,10000,...",21.23,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00
4,NX_426260,PH-20240101-07,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:10.161 +0000,2023-12-31 23:00:00+00:00,"{'X5384463871','X5385167819','X5385169379','X5...","{'X5385370820','X5385371596','X5385371593','X5...","{1748,1748,1748,1748,1746,1520,1500,1500,1362,...","{2039,2089,2090,2262,2462,2562,2670,2700,2710,...","{4900,1000,1000,500,5000,1000,2000,1300,5000,5...","{500,1000,10000,15000,10000,1000,5000,5000,100...",22.62,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10768015,NX_496910,PH-20250101-20,2025-01-01 18:00:00+00:00,2025-01-01 19:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:44.288 +0000,2024-12-31 22:59:00+00:00,"{'X10233448499','X10234051602','X10234052934',...","{'X10232980063','X10228851570','X10232667440',...","{980,828,828,809,743,721,721,658,575,545}","{2775,2780,2780,2810,2810,2870,2900,3500}","{3900,4000,5000,5000,5000,5000,4000,800,2000,200}","{200,8000,5000,5000,5000,10000,2700,10000}",NaN,NaT,NaT
10768016,NX_496918,PH-20250101-21,2025-01-01 19:00:00+00:00,2025-01-01 20:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:48.436 +0000,2024-12-31 22:59:00+00:00,"{'X10233744528','X10233744531','X10233744825',...","{'X10228851573','X10229998186','X10229998185',...","{513,513,509,500,498,498,497,394,377,376}","{2780,2780,2810,2810,2820,2870,2900,2900,2950,...","{5000,5000,5000,3900,5000,5000,300,2500,5000,6...","{6400,5000,5000,5000,10000,10000,1900,5000,700...",NaN,NaT,NaT
10768017,NX_496925,PH-20250101-22,2025-01-01 20:00:00+00:00,2025-01-01 21:00:00+00:00,10,N_2,NO3,2024-12-31 23:43:52.589 +0000,2024-12-31 22:59:00+00:00,"{'X10234238392','X10234238451','X10234238459',...",

## Remove rows where DA price is NaN
DA prices go a little bit further than the orderbook. Remove these

In [13]:
df = df.dropna(subset=['NO3 Price (EUR)'])

## Write merged orderbook to parquet

In [14]:
df.to_parquet('Data/merged_orderbook.parquet')

## Start here

In [3]:
df = pd.read_parquet('Data/merged_orderbook.parquet')

In [25]:
df

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,orderId_ask,orderPrice_bid,orderPrice_ask,orderQty_bid,orderQty_ask,NO3 Price (EUR),Delivery Start (UTC),Delivery End (UTC)
index,,,,,,,,,,,,,,,,,,
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...","{'X5385371204','X5385374175','X5385353666','X5...","{2247,2100,2000,2000,1751,1443,1212,1205,1205,...","{2350,2378,2409,2497,2514,2559,2590,2600,2657,...","{500,5000,800,1600,1000,700,5000,5200,800,2200}","{10000,2000,5000,1000,200,500,300,10000,4000,500}",26.66,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00
1,NX_426238,PH-20240101-04,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:56.947 +0000,2023-12-31 23:00:00+00:00,"{'X5385372330','X5385372495','X5385371311','X5...","{'X5385372323','X5385372333','X5385373948','X5...","{2061,2061,2051,2000,2000,2000,1920,1467,1000,...","{2251,2251,2262,2299,2300,2433,2447,2448,2551,...","{1900,3000,100,5000,800,1500,400,500,1000,1000}","{200,500,200,3000,10000,500,3600,15000,4000,4000}",24.48,2024-01-01 02:00:00+00:00,2024-01-01 03:00:00+00:00
2,NX_426245,PH-20240101-05,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:00.755 +0000,2023-12-31 23:00:00+00:00,"{'X5385359141','X5385367235','X5385367212','X5...","{'X5385364822','X5385364828','X5385364821','X5...","{2000,1950,1927,1917,1916,1320,1319,1319,1319,...","{2127,2127,2129,2150,2200,2400,2400,2401,2401,...","{100,100,1200,9000,2000,4500,1600,500,1000,1000}","{200,5000,400,10000,2000,4000,5000,500,15000,400}",24.01,2024-01-01 03:00:00+00:00,2024-01-01 04:00:00+00:00
3,NX_426252,PH-20240101-06,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:06.307 +0000,2023-12-31 23:00:00+00:00,"{'X5385371584','X5385371589','X5385371590','X5...","{'X5385363190','X5385363118','X5383614807','X5...","{1574,1574,1574,1574,1574,1574,1424,1423,1000,...","{1798,1800,1875,2050,2123,2423,2504,2550,2670,...","{1500,5000,1000,2000,1000,5000,2000,5000,1000,...","{10000,10000,10000,5000,15000,1000,1900,10000,...",21.23,2024-01-01 04:00:00+00:00,2024-01-01 05:00:00+00:00
4,NX_426260,PH-20240101-07,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00,10,N_2,NO3,2024-03-06 10:25:10.161 +0000,2023-12-31 23:00:00+00:00,"{'X5384463871','X5385167819','X5385169379','X5...","{'X5385370820','X5385371596','X5385371593','X5...","{1748,1748,1748,1748,1746,1520,1500,1500,1362,...","{2039,2089,2090,2262,2462,2562,2670,2700,2710,...","{4900,1000,1000,500,5000,1000,2000,1300,5000,5...","{500,1000,10000,15000,10000,1000,5000,5000,100...",22.62,2024-01-01 05:00:00+00:00,2024-01-01 06:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10765075,NX_496748,PH-20241231-24,2024-12-31 22:00:00+00:00,2024-12-31 23:00:00+00:00,10,N_2,NO3,2024-12-31 21:39:10.554 +0000,2024-12-31 20:55:00+00:00,"{'X10224735588','X10231756122','X10226066681',...","{'X10226066683','X10232275812','X10223438983',...","{1262,1100,885,885,650,640,586,371,370,340}","{1685,1889,1973,2260,2310,2535,2535,2535,2700,...","{4300,6900,5200,1700,1000,200,10000,100,5300,200}","{1500,200,300,2000,200,5000,5000,5000,5000,5000}",12.85,2024-12-31 22:00:00+00:00,2024-12-31 23:00:00+00:00
10765100,NX_496748,PH-20241231-24,2024-12-31 22:00:00+00:00,2024-12-31 23:00:00+00:00,10,N_2,NO3,2024-12-31 21:39:10.554 +0000,2024-12-31 20:56:00+00:00,"{'X10224735588','X10231756122','X10226066681',...","{'X10226066683','X10232275812','X10223438983',...","{1262,1100,885,885,690,680,586,371,370,344}","{1685,1889,1973,2250,2270,2531,2531,2531,2700,...","{4300,6900,5200,1700,1000,200,10000,100,5300,1...","{1500,200,300,2000,200,5000,5000,5000,5000,5000}",12.85,2024-12-31 22:00:00+00:00,2024-12-31 23:00:00+00:00
10765125,NX_496748,PH-20241231-24,2024-12-31 22:00:00+00:00,2024-12-31 2

In [21]:
df.dtypes

contractId                           object
contractName                         object
dlvryStart              datetime64[ns, UTC]
dlvryEnd                datetime64[ns, UTC]
dlvryAreaId                           int64
marketId                             object
area                                 object
recorded                             object
time                    datetime64[ns, UTC]
orderId_bid                          object
orderId_ask                          object
orderPrice_bid                       object
orderPrice_ask                       object
orderQty_bid                         object
orderQty_ask                         object
NO3 Price (EUR)                     float64
Delivery Start (UTC)    datetime64[ns, UTC]
Delivery End (UTC)      datetime64[ns, UTC]
dtype: object

## Formatting the DF

In [92]:
# Set the delivery hour to be the index. This will probably not be needed
def setIndex(df):
    df = df.set_index(['dlvryStart', 'time'])
    return df

# Sort by dlvryStart and time. Meaning that we get dlvryStart at e.g. 23 with time=14.01, 14.02 etc before dlvryStart at 00 with time=14.01, 14.02 etc
def sortByDlvryStart(df):
    # Sort by time within each index group
    df = df.sort_values(by=['dlvryStart', 'time'])
    df = df.groupby(df['dlvryStart'], group_keys=False).apply(lambda x: x.sort_values(by='time'))
    return df

# Drop unnecessary columns
def dropColumns(df):
    df = df.drop(columns=['contractId', 'contractName', 'dlvryEnd', 'dlvryAreaId', 'marketId', 'area', 'recorded', 'Delivery Start (UTC)', 'Delivery End (UTC)'])
    return df
'''
# Used in the apply_process_string function
def process_string(input_str):
    # Check if the string is empty or contains only curly braces
    if input_str in ['{}', '']:
        return []   
    else:
        # Remove the curly braces, split by commas, and convert to float
        return [float(x) for x in input_str[1:-1].split(',')]
'''

# Used in the apply_process_string function
def process_string(input_str):
    # Check if the string is empty or contains only curly braces
    if input_str in ['{}', '']:
        return []   
    else:
        # Direct string slicing and list comprehension in one pass
        content = input_str[1:-1]
        if not content:
            return []
    
        # Split and convert in one operation
        return [float(x) for x in content.split(',')]

# Converts the string representation of a list to a list of floats
def apply_process_string(df):
    cols = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask']
    for col in cols:
        # Apply the process_string function to each element in the column
        df[col] = df[col].apply(lambda x: process_string(x) if isinstance(x, str) else x)
    return df

def retrieveBestOffers(df):
    columns = ['orderPrice_bid', 'orderQty_bid', 'orderPrice_ask', 'orderQty_ask']
    for col in columns:
        df[col + '_best'] = df[col].apply(lambda x: x[0] if len(x) > 0 else pd.NA)
    # Check if there are any bids/asks
    df['exists_bid'] = df['orderPrice_bid'].apply(lambda x: 1 if len(x) > 0 else 0)
    df['exists_ask'] = df['orderPrice_ask'].apply(lambda x: 1 if len(x) > 0 else 0)
    return df

def fixPriceVolumeMagnitude(df):
    # Define the columns to be processed
    #price_cols = ['orderPrice_bid', 'orderPrice_ask', 'orderPrice_bid_best', 'orderPrice_ask_best']
    #qty_cols = ['orderQty_bid', 'orderQty_ask', 'orderQty_bid_best', 'orderQty_ask_best']
    price_cols = ['orderPrice_bid_best', 'orderPrice_ask_best']
    qty_cols = ['orderQty_bid_best', 'orderQty_ask_best']
    
    for col in price_cols:
        df[col] = df[col].apply(lambda x: x / 100 if pd.notna(x) else x)

    for col in qty_cols:
        df[col] = df[col].apply(lambda x: x / 1000 if pd.notna(x) else x)

    return df

# Rename column names for better readability
def changeName(df):
    df = df.rename(columns={'NO3 Price (EUR)': 'DA_price'})
    return df

# Split delivery hour and time to month, day, hour etc
def createTimeColumns(df):
    df['dlvryMonth'] = df['dlvryStart'].dt.month
    df['dlvryDay'] = df['dlvryStart'].dt.day
    df['dlvryHour'] = df['dlvryStart'].dt.hour
    df['dlvryWeekday'] = df['dlvryStart'].dt.weekday

    df['timeMonth'] = df['time'].dt.month
    df['timeDay'] = df['time'].dt.day
    df['timeHour'] = df['time'].dt.hour
    df['timeWeekday'] = df['time'].dt.weekday
    df['timeMinute'] = df['time'].dt.minute
    return df

# For each row it find the best price that appears later in the LOB for the same delivery hour.
def findFutureBestOffers(df):
    # Replace NaNs with -inf for bids and +inf for asks, because there is some issues with compariosn of NaNs to ordinary values
    df['orderPrice_bid_best'] = df['orderPrice_bid_best'].fillna(-np.inf)
    df['orderPrice_ask_best'] = df['orderPrice_ask_best'].fillna(np.inf)

    # Initalize new columns
    new_cols = ['better_bid_later', 'better_ask_later', 'future_best_bid', 'future_best_ask']

    delivery_hours = df.groupby('dlvryStart')
    for _, group in delivery_hours:
        reverse_group = group.iloc[::-1].copy() # Reverses the order
        reverse_group['cum_max_bid'] = reverse_group['orderPrice_bid_best'].cummax() # Finds the best price seen so far
        reverse_group['cum_min_ask'] = reverse_group['orderPrice_ask_best'].cummin() # Finds the best price seen so far

        # Restore the original order
        forward_group = reverse_group.iloc[::-1].copy()

        # Set future best prices
        forward_group['future_best_bid'] = forward_group['cum_max_bid'].shift(-1)
        forward_group['future_best_ask'] = forward_group['cum_min_ask'].shift(-1)
        
        # Compare current prices with future best prices
        forward_group['better_bid_later'] = (
            forward_group['cum_max_bid'].shift(-1) > forward_group['orderPrice_bid_best']
        ).astype(int)

        forward_group['better_ask_later'] = (
            forward_group['cum_min_ask'].shift(-1) < forward_group['orderPrice_ask_best']
        ).astype(int)

        # Convert infinities back to NaN for clean output
        forward_group['future_best_bid'] = forward_group['future_best_bid'].replace(-np.inf, pd.NA)
        forward_group['future_best_ask'] = forward_group['future_best_ask'].replace(np.inf, pd.NA)
        
        # Update original dataframe
        df.loc[forward_group.index, new_cols] = forward_group[new_cols]
        
    # Restore original NaN values in source columns
    df['orderPrice_bid_best'] = df['orderPrice_bid_best'].replace(-np.inf, np.nan)
    df['orderPrice_ask_best'] = df['orderPrice_ask_best'].replace(np.inf, np.nan)
    
    return df

# For each delivery hour. I want to find the best far price, aka the best price from market opening until six hours before delivery. 
def findBestFarPrice(df):
    df['best_far_bid'] = pd.NA
    df['best_far_ask'] = pd.NA
    groups = df.groupby('dlvryStart')
    for _, group in groups:
        # Find the cutoff timestamp (6 hours before delivery)
        cutoff_time = group['dlvryStart'].iloc[0] - pd.Timedelta(hours=6)

        # Filter data up to cutoff time
        far_market_data = group[group['time'] <= cutoff_time]

        # Calculate the best far prices
        best_far_bid = far_market_data['orderPrice_bid_best'].max(skipna=True)
        best_far_ask = far_market_data['orderPrice_ask_best'].min(skipna=True)

        # Assign values to the original DataFrame
        df.loc[group.index, 'best_far_bid'] = best_far_bid
        df.loc[group.index, 'best_far_ask'] = best_far_ask

    return df

# Create a column that shows time until delivery in minutes. Might capture something
def createTimeUntilDeliveryCol(df):
    df['minutesFromDelivery'] = (df['dlvryStart'] - df['time']).dt.total_seconds() / 60 # Both are datetime objects
    return df

# Drop the last minute of trading. We cant take trades after the last minute of trading, so it makes sense to exclude this row. 
def dropLastMinuteOfTrading(df):
    return df.groupby('dlvryStart', group_keys=False).apply(lambda group: group.iloc[:-1])

# Create intervals. Only keep rows every 30 minutes when more than 6 hours from delivery. Use 15-minute intervals when 2-6 hours from delivery. Use 5-minute intervals when less than 2 hours from delivery
def createIntervals(df):
    df['hoursUntilDelivery'] = df['minutesFromDelivery'] / 60
    
     # Filter rows based on time_to_delivery using boolean indexing
    filtered_df = df[
        ((df['hoursUntilDelivery'] > 6) & (df['time'].dt.minute % 30 == 0)) |  # Every 30 min when > 6h
        ((df['hoursUntilDelivery'] >= 2) & (df['hoursUntilDelivery'] <= 6) & (df['time'].dt.minute % 15 == 0)) |  # Every 15 min when 2-6h
        ((df['hoursUntilDelivery'] < 2) & (df['time'].dt.minute % 5 == 0))  # Every 5 min when < 2h
    ]
    
    return filtered_df

def create24hLag(df):
    # Create columns for 24h ago
    df['dlvryStart_24h_ago'] = df['dlvryStart'] - pd.Timedelta(days=1)
    df['time_24h_ago'] = df['time'] - pd.Timedelta(days=1)
    
    # Merge to get values from 24h ago
    df_with_lags = pd.merge(
        df,
        df[['dlvryStart', 'time', 'future_best_bid', 'future_best_ask']],
        left_on=['dlvryStart_24h_ago', 'time_24h_ago'],
        right_on=['dlvryStart', 'time'],
        how='left',
        suffixes=('', '_24h_ago')
    )
    
    # Drop temporary columns
    df_with_lags = df_with_lags.drop(['dlvryStart_24h_ago', 'time_24h_ago', 'dlvryStart_24h_ago'], axis=1)
    
    return df_with_lags




 

In [ ]:
#df = setIndex(df) # Maybe not needed
#df = dropColumns(df) # Maybe better to just exclude in the FEATURES

In [35]:
df['future_best_bid'].tail(5)

10765075    12.62
10765100    12.62
10765125    11.01
10765150    11.01
10765175      NaN
Name: future_best_bid, dtype: object

In [112]:
#df = createTimeColumns(df) # Creates columns out of dlvryStart and time
#df = sortByDlvryStart(df) # Sorts the dataframe by dlvyStart and then time
#df = apply_process_string(df) # Converts orderPrice_bid etc from '{}' to list []. Easier to extract the elements then
#df = retrieveBestOffers(df) # Gets the best prices and the corresponding quantities and stores them in new columns
#df = fixPriceVolumeMagnitude(df) # Divde prices by 100 and quantities by 1000 to get euro/MWh and MWh
#df = changeName(df) # Changes the name of columns for better readibility
#df = findBestFarPrice(df) # Finds the best price from market opening until six hours before delivery
#df = findFutureBestOffers(df) # Finds the best best price that appears later in the orderbook for the same delivery hour
#df = createTimeUntilDeliveryCol(df)
#df = dropLastMinuteOfTrading(df)
#df = createIntervals(df)
#df = create24hLag(df)

df.to_parquet('Data/formatted_orderbook.parquet')
#df = pd.read_parquet('Data/formatted_orderbook.parquet')





In [111]:
df.head(5)

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask,minutesFromDelivery,hoursUntilDelivery,future_best_bid_24h_ago,future_best_ask_24h_ago
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,NaN,NaN,1.0,0.0,30.02,23.80,120.0,2.000000,NaN,NaN
1,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:05:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,NaN,NaN,1.0,1.0,30.02,23.80,115.0,1.916667,NaN,NaN
2,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:10:00+00:00,"{'X5385417804','X5385417826','X5385422848','X5...",...,NaN,NaN,1.0,1.0,30.02,23.80,110.0,1.833333,NaN,NaN
3,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:15:00+00:00,"{'X5385443452','X5385431716','X5385444852','X5...",...,NaN,NaN,1.0,1.0,30.02,25.29,105.0,1.750000,NaN,NaN
4,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:20:00+00:00,"{'X5385443452','X5385431716','X5385465106','X5...",...,NaN,NaN,1.0,1.0,30.02,25.29,100.0,1.666667,NaN,NaN


In [102]:
df_copy[df_copy['dlvryStart'] == '2024-01-02 23:00:00+00:00'][['dlvryStart', 'time', 'orderPrice_bid', 'orderPrice_bid_best', 'orderPrice_ask', 'exists_bid', 'future_best_bid', 'future_best_bid_24h_ago']].tail(5)

,dlvryStart,time,orderPrice_bid,orderPrice_bid_best,orderPrice_ask,exists_bid,future_best_bid,future_best_bid_24h_ago
2329,2024-01-02 23:00:00+00:00,2024-01-02 21:35:00+00:00,"[4500.0, 4320.0, 4320.0, 4088.0, 4079.0, 4079....",45.00,"[4800.0, 4800.0, 4870.0, 4870.0, 4870.0, 5200....",1,48.3,38.92
2330,2024-01-02 23:00:00+00:00,2024-01-02 21:40:00+00:00,"[4500.0, 4380.0, 4380.0, 4088.0, 4079.0, 4079....",45.00,"[4800.0, 4800.0, 4870.0, 4870.0, 4870.0, 5200....",1,48.3,34.36
2331,2024-01-02 23:00:00+00:00,2024-01-02 21:45:00+00:00,"[4500.0, 4380.0, 4380.0, 4360.0, 4360.0, 4360....",45.00,"[4870.0, 4870.0, 4870.0, 4896.0, 4999.0, 5200....",1,48.3,34.36
2332,2024-01-02 23:00:00+00:00,2024-01-02 21:50:00+00:00,"[4637.0, 4500.0, 4497.0, 4497.0, 4380.0, 4380....",46.37,"[4867.0, 4870.0, 4870.0, 4870.0, 5200.0, 5250....",1,48.3,34.36
2333,2024-01-02 23:00:00+00:00,2024-01-02 21:55:00+00:00,"[4525.0, 4500.0, 4380.0, 4380.0, 4352.0, 4105....",45.25,"[4895.0, 5200.0, 5250.0, 5250.0, 5290.0, 5290....",1,48.3,30.10


In [95]:
df_copy[df_copy['dlvryStart'] == '2024-08-02 01:00:00+00:00'][['dlvryStart', 'time', 'orderPrice_bid', 'orderPrice_bid_best', 'orderPrice_ask', 'exists_bid', 'future_best_bid', 'future_best_bid_24h_ago']].tail(5)

,dlvryStart,time,orderPrice_bid,orderPrice_bid_best,orderPrice_ask,exists_bid,future_best_bid,future_best_bid_24h_ago
301953,2024-08-02 01:00:00+00:00,2024-08-01 23:35:00+00:00,"[2085.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085....",20.85,"[2515.0, 2600.0, 3450.0, 4450.0, 5450.0, 5800....",1,20.85,12.6
301954,2024-08-02 01:00:00+00:00,2024-08-01 23:40:00+00:00,"[2085.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085....",20.85,"[2515.0, 2600.0, 3450.0, 4450.0, 5450.0, 5800....",1,20.85,12.6
301955,2024-08-02 01:00:00+00:00,2024-08-01 23:45:00+00:00,"[2085.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085....",20.85,"[2519.0, 2600.0, 3450.0, 4450.0, 5450.0, 5800....",1,20.85,12.6
301956,2024-08-02 01:00:00+00:00,2024-08-01 23:50:00+00:00,"[2085.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085....",20.85,"[2519.0, 2600.0, 3450.0, 4450.0, 5450.0, 5800....",1,20.85,12.6
301957,2024-08-02 01:00:00+00:00,2024-08-01 23:55:00+00:00,"[2085.0, 2085.0, 2085.0, 2085.0, 2085.0, 2085....",20.85,"[2519.0, 2600.0, 3450.0, 4450.0, 5450.0, 5800....",1,20.85,11.0


In [5]:
df.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'DA_price', 'Delivery Start (UTC)', 'Delivery End (UTC)', 'dlvryMonth',
       'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay',
       'timeHour', 'timeWeekday', 'timeMinute', 'orderPrice_bid_best',
       'orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best',
       'exists_bid', 'exists_ask', 'best_far_bid', 'best_far_ask',
       'better_bid_later', 'better_ask_later', 'future_best_bid',
       'future_best_ask', 'minutesFromDelivery'],
      dtype='object')

In [38]:
df[df['dlvryStart'] == '2024-01-01 01:00:00+00:00'][['dlvryStart', 'time', 'orderPrice_bid', 'orderPrice_bid_best', 'orderPrice_ask', 'orderPrice_ask_best', 'best_far_bid', 'best_far_ask', 'future_best_bid', 'future_best_ask', 'better_bid_later', 'better_ask_later']].iloc[0:10]

,dlvryStart,time,orderPrice_bid,orderPrice_bid_best,orderPrice_ask,orderPrice_ask_best,best_far_bid,best_far_ask,future_best_bid,future_best_ask,better_bid_later,better_ask_later
0,2024-01-01 01:00:00+00:00,2023-12-31 23:00:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1443....",22.47,"[2350.0, 2378.0, 2409.0, 2497.0, 2514.0, 2559....",23.50,NaN,NaN,30.02,23.8,1.0,0.0
22,2024-01-01 01:00:00+00:00,2023-12-31 23:01:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1586....",22.47,"[2412.0, 2470.0, 2514.0, 2520.0, 2559.0, 2559....",24.12,NaN,NaN,30.02,23.8,1.0,1.0
44,2024-01-01 01:00:00+00:00,2023-12-31 23:02:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1662....",22.47,"[2420.0, 2459.0, 2469.0, 2477.0, 2514.0, 2520....",24.20,NaN,NaN,30.02,23.8,1.0,1.0
66,2024-01-01 01:00:00+00:00,2023-12-31 23:03:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1591....",22.47,"[2459.0, 2470.0, 2477.0, 2514.0, 2520.0, 2559....",24.59,NaN,NaN,30.02,23.8,1.0,1.0
88,2024-01-01 01:00:00+00:00,2023-12-31 23:04:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1581....",22.47,"[2429.0, 2450.0, 2459.0, 2477.0, 2514.0, 2559....",24.29,NaN,NaN,30.02,23.8,1.0,1.0
110,2024-01-01 01:00:00+00:00,2023-12-31 23:05:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1638....",22.47,"[2429.0, 2450.0, 2459.0, 2477.0, 2514.0, 2559....",24.29,NaN,NaN,30.02,23.8,1.0,1.0
132,2024-01-01 01:00:00+00:00,2023-12-31 23:06:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1751.0, 1666....",22.47,"[2429.0, 2450.0, 2459.0, 2477.0, 2514.0, 2559....",24.29,NaN,NaN,30.02,23.8,1.0,1.0
154,2024-01-01 01:00:00+00:00,2023-12-31 23:07:00+00:00,"[2247.0, 2100.0, 2000.0, 2000.0, 1752.0, 1751....",22.47,"[2459.0, 2477.0, 2480.0, 2514.0, 2559.0, 2559....",24.59,NaN,NaN,30.02,23.8,1.0,1.0
176,2024-01-01 01:00:00+00:00,2023-12-31 23:08:00+00:00,"[2000.0, 2000.0, 2000.0, 1946.0, 1779.0, 1751....",20.00,"[2400.0, 2429.0, 2473.0, 2479.0, 2559.0, 2600....",24.00,NaN,NaN,30.02,23.8,1.0,1.0
198,2024-01-01 01:00:00+00:00,2023-12-31 23:09:00+00:00,"[2088.0, 2088.0, 1871.0, 1751.0, 1500.0, 1475....",20.88,"[2390.0, 2440.0, 2551.0, 2559.0, 2600.0, 2646....",23.90,NaN,NaN,30.02,23.8,1.0,1.0


## Train/Test split

In [113]:
# Just create a copy of the dataframe
df_copy = df.copy()
train, test = train_test_split(df_copy, test_size=0.2, shuffle=False)

In [7]:
train

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,orderQty_ask_best,exists_bid,exists_ask,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask,minutesFromDelivery
0,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:00:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,10.0,1,1,NaN,NaN,1.0,0.0,30.02,23.8,120.0
22,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:01:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,0.5,1,1,NaN,NaN,1.0,1.0,30.02,23.8,119.0
44,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:02:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,9.1,1,1,NaN,NaN,1.0,1.0,30.02,23.8,118.0
66,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:03:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,0.3,1,1,NaN,NaN,1.0,1.0,30.02,23.8,117.0
88,NX_426231,PH-20240101-03,2024-01-01 01:00:00+00:00,2024-01-01 02:00:00+00:00,10,N_2,NO3,2024-03-06 10:24:54.175 +0000,2023-12-31 23:04:00+00:00,"{'X5385339387','X5385339295','X5385350510','X5...",...,0.6,1,1,NaN,NaN,1.0,1.0,30.02,23.8,116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8600724,NX_482960,PH-20241020-09,2024-10-20 06:00:00+00:00,2024-10-20 07:00:00+00:00,10,N_2,NO3,2024-10-19 15:44:57.884 +0000,2024-10-19 15:19:00+00:00,"{'X9024190130','X9024158526','X9025800168','X9...",...,5.0,1,1,5.0,9.0,1.0,1.0,5.20,8.8,881.0
8600753,NX_482960,PH-20241020-09,2024-10-20 06:00:00+00:00,2024-10-20 07:00:00+00:00,10,N_2,NO3,2024-10-19 15:44:57.884 +0000,2024-10-19 15:20:00+00:00,"{'X9024190130','X9024158526','X9025817486','X9...",...,5.0,1,1,5.0,9.0,1.0,1.0,5.20,8.8,880.0
8600782,NX_482960,PH-20241020-09,2024-10-20 06:00:00+00:00,2024-10-20 07:00:00+00:00,10,N_2,NO3,2024-10-19 15:44:57.884 +0000,2024-10-19 15:21:00+00:00,"{'X9024190130','X9024158526','X9025832274','X9...",...,5.0,1,1,5.0,9.0,1.0,1.0,5.20,8.8,879.0
8600811,NX_482960,PH-20241020-09,2024-10-20 06:00:00+00:00,2024-10-20 07:00:00+00:00,10,N_2,NO3,2024-10-19 15:44:57.884 +0000,2024-10-19 15:22:00+00:00,"{'X9024190130','X9024158526','X9025843909','X9...",...,5.0,1,1,5.0,9.0,1.0,1.0,5.20,8.8,878.0


In [55]:
def filter_random_test_set(test):
    # Ensure 'dlvryStart' is in datetime format
    test['dlvryStart'] = pd.to_datetime(test['dlvryStart'])
    # Extract delivery hour from the 'dlvryStart' column
    test['dlvryHour'] = test['dlvryStart'].dt.hour
    # Group by delivery hour and sample one random row per group
    filtered_test = test.groupby('dlvryHour', group_keys=False).apply(lambda x: x.sample(1))
    # Drop the helper column 'dlvryHour' after filtering
    filtered_test = filtered_test.drop(columns=['dlvryHour'])
    return filtered_test

# Apply the filtering function to the test set
test_filtered = filter_random_test_set(test)
print(test)

                        dlvryStart                      time  \
8600869  2024-10-20 06:00:00+00:00 2024-10-19 15:24:00+00:00   
8600898  2024-10-20 06:00:00+00:00 2024-10-19 15:25:00+00:00   
8600927  2024-10-20 06:00:00+00:00 2024-10-19 15:26:00+00:00   
8600956  2024-10-20 06:00:00+00:00 2024-10-19 15:27:00+00:00   
8600985  2024-10-20 06:00:00+00:00 2024-10-19 15:28:00+00:00   
...                            ...                       ...   
10765075 2024-12-31 22:00:00+00:00 2024-12-31 20:55:00+00:00   
10765100 2024-12-31 22:00:00+00:00 2024-12-31 20:56:00+00:00   
10765125 2024-12-31 22:00:00+00:00 2024-12-31 20:57:00+00:00   
10765150 2024-12-31 22:00:00+00:00 2024-12-31 20:58:00+00:00   
10765175 2024-12-31 22:00:00+00:00 2024-12-31 20:59:00+00:00   

                                                orderId_bid  \
8600869   {'X9024190130','X9024158526','X9025865742','X9...   
8600898   {'X9024190130','X9024158526','X9025877707','X9...   
8600927   {'X9024190130','X9024158526','X9

In [114]:
df_copy.columns

Index(['contractId', 'contractName', 'dlvryStart', 'dlvryEnd', 'dlvryAreaId',
       'marketId', 'area', 'recorded', 'time', 'orderId_bid', 'orderId_ask',
       'orderPrice_bid', 'orderPrice_ask', 'orderQty_bid', 'orderQty_ask',
       'DA_price', 'Delivery Start (UTC)', 'Delivery End (UTC)', 'dlvryMonth',
       'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay',
       'timeHour', 'timeWeekday', 'timeMinute', 'orderPrice_bid_best',
       'orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best',
       'exists_bid', 'exists_ask', 'best_far_bid', 'best_far_ask',
       'better_bid_later', 'better_ask_later', 'future_best_bid',
       'future_best_ask', 'minutesFromDelivery', 'hoursUntilDelivery',
       'future_best_bid_24h_ago', 'future_best_ask_24h_ago'],
      dtype='object')

In [115]:
# I think it is better to train a separate model for bid and ask
# I dont think xgboost handles lists as input, so we need to convert the lists to separate columns or just drop them
FEATURES_BID = ['orderPrice_bid_best','orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best', 'best_far_bid', 'best_far_ask', 'DA_price', 'dlvryMonth', 'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour', 'timeWeekday', 'timeMinute', 'minutesFromDelivery', 'exists_bid', 'exists_ask', 'future_best_bid_24h_ago', 'future_best_ask_24h_ago']
FEATURES_ASK = ['orderPrice_bid_best','orderQty_bid_best', 'orderPrice_ask_best', 'orderQty_ask_best', 'best_far_bid', 'best_far_ask', 'DA_price', 'dlvryMonth', 'dlvryDay', 'dlvryHour', 'dlvryWeekday', 'timeMonth', 'timeDay', 'timeHour', 'timeWeekday', 'timeMinute', 'minutesFromDelivery', 'exists_bid', 'exists_ask', 'future_best_bid_24h_ago', 'future_best_ask_24h_ago']
TARGET_BID = ['future_best_bid']
TARGET_ASK = ['future_best_ask']

In [116]:
x_train_bid = train[FEATURES_BID]
y_train_bid = train[TARGET_BID]
x_test_bid = test[FEATURES_BID]
y_test_bid = test[TARGET_BID]

x_train_ask = train[FEATURES_ASK]
y_train_ask = train[TARGET_ASK]
x_test_ask = test[FEATURES_ASK]
y_test_ask = test[TARGET_ASK]

## Handling Nan and Inf
If there are no future orders, they get assigned a NaN value. This is a problem for Xgboost, meaning we must convert these NaN values to something. We try to replace NaN with a special value that indicates 'no future orders' and treat it as a valid target value. Edit: almost all of these rows where the future best order is NaN is the last minute of trading, so it makes sense. Should find out when there are no more better trades. Could be useful in the data analysis as well.

In [9]:
na_rows = df[df['future_best_bid'].isna()]

In [14]:
na_rows.iloc[15:30]

,contractId,contractName,dlvryStart,dlvryEnd,dlvryAreaId,marketId,area,recorded,time,orderId_bid,...,orderPrice_ask_best,orderQty_ask_best,exists_bid,exists_ask,best_far_bid,best_far_ask,better_bid_later,better_ask_later,future_best_bid,future_best_ask
1879128,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:32:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879152,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:33:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879176,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:34:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879200,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:35:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879224,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:36:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879248,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:37:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879272,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:38:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879296,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:39:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879320,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:40:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0
1879344,NX_438970,PH-20240305-01,2024-03-04 23:00:00+00:00,2024-03-05 00:00:00+00:00,10,N_2,NO3,2024-03-06 12:11:27.102 +0000,2024-03-04 21:41:00+00:00,{},...,50.0,5.0,0,1,48.0,55.11,0.0,0.0,NaN,50.0


In [11]:
print(len(na_rows))

4061


In [118]:
y_train_bid = y_train_bid.fillna(-999)
y_test_bid = y_test_bid.fillna(-999)

y_train_ask = y_train_ask.fillna(999)
y_test_ask = y_test_ask.fillna(999)

In [119]:
# Check and clean for NaN in y_train and y_test
# Bids
print("NaN values in y_train_bid before cleaning:", y_train_bid.isna().sum().sum())
print("NaN values in y_test_bid before cleaning:", y_test_bid.isna().sum().sum())

# Asks
print("NaN values in y_train_ask before cleaning:", y_train_ask.isna().sum().sum())
print("NaN values in y_test_ask before cleaning:", y_test_ask.isna().sum().sum())
'''
print("NaN values in x_train before cleaning:", x_train_bid_imputed.isna().sum().sum())
print("NaN values in x_test before cleaning:", x_test_bid.isna().sum().sum())
'''

NaN values in y_train_bid before cleaning: 0
NaN values in y_test_bid before cleaning: 0
NaN values in y_train_ask before cleaning: 0
NaN values in y_test_ask before cleaning: 0


'\nprint("NaN values in x_train before cleaning:", x_train_bid_imputed.isna().sum().sum())\nprint("NaN values in x_test before cleaning:", x_test_bid.isna().sum().sum())\n'

In [20]:
# Ensure there are no inf values left in features
print("Infinite values in x_train:", np.isinf(x_train_bid).sum().sum())
print("Infinite values in x_test:", np.isinf(x_test_bid).sum().sum())

Infinite values in x_train: 0
Infinite values in x_test: 0


## Plot the data
Need to plot the data. Maybe use something like sns.pairplot like in the tennis video. Variables plotted against each other

## Model bid

In [120]:
# Create model instance
model_bid = XGBRegressor(n_estimators=1000, early_stopping_rounds=50, learning_rate=0.01)
# Fit model
model_bid.fit(x_train_bid, y_train_bid, eval_set=[(x_train_bid, y_train_bid), (x_test_bid, y_test_bid)], verbose=True)
# Make predictions  
preds_bid = model_bid.predict(x_test_bid)

[0]	validation_0-rmse:37.55976	validation_1-rmse:39.91144
[1]	validation_0-rmse:37.25711	validation_1-rmse:39.74781
[2]	validation_0-rmse:36.95801	validation_1-rmse:39.58648
[3]	validation_0-rmse:36.69169	validation_1-rmse:39.43126
[4]	validation_0-rmse:36.39892	validation_1-rmse:39.27541
[5]	validation_0-rmse:36.10980	validation_1-rmse:39.12479
[6]	validation_0-rmse:35.85262	validation_1-rmse:38.97813
[7]	validation_0-rmse:35.56949	validation_1-rmse:38.82928
[8]	validation_0-rmse:35.28909	validation_1-rmse:38.68503
[9]	validation_0-rmse:35.04027	validation_1-rmse:38.54754
[10]	validation_0-rmse:34.76663	validation_1-rmse:38.41746
[11]	validation_0-rmse:34.49617	validation_1-rmse:38.29006
[12]	validation_0-rmse:34.25571	validation_1-rmse:38.16066
[13]	validation_0-rmse:33.99111	validation_1-rmse:38.02774
[14]	validation_0-rmse:33.75554	validation_1-rmse:37.90380
[15]	validation_0-rmse:33.49626	validation_1-rmse:37.78530
[16]	validation_0-rmse:33.26628	validation_1-rmse:37.66582
[17]	va

In [121]:
results_bid = x_test_bid.copy()
results_bid['preds'] = preds_bid
results_bid['actual'] = y_test_bid

In [123]:
results_bid.head(10)

,orderPrice_bid_best,orderQty_bid_best,orderPrice_ask_best,orderQty_ask_best,best_far_bid,best_far_ask,DA_price,dlvryMonth,dlvryDay,dlvryHour,...,timeHour,timeWeekday,timeMinute,minutesFromDelivery,exists_bid,exists_ask,future_best_bid_24h_ago,future_best_ask_24h_ago,preds,actual
412691,4.09,5.0,7.90,0.4,4.0,7.0,4.49,10,20,4,...,1,6,15,165.0,1,1,11.99,6.99,7.466879,4.09
412692,4.09,5.0,7.30,0.7,4.0,7.0,4.49,10,20,4,...,1,6,30,150.0,1,1,11.99,6.99,7.336144,4.09
412693,4.00,5.0,6.87,1.0,4.0,7.0,4.49,10,20,4,...,1,6,45,135.0,1,1,11.99,6.99,7.326568,4.09
412694,4.09,5.0,6.96,0.9,4.0,7.0,4.49,10,20,4,...,2,6,0,120.0,1,1,11.99,9.19,7.243277,4.09
412695,4.09,5.0,6.96,1.0,4.0,7.0,4.49,10,20,4,...,2,6,5,115.0,1,1,11.99,9.49,7.229598,4.09
412696,4.09,5.0,7.54,1.4,4.0,7.0,4.49,10,20,4,...,2,6,10,110.0,1,1,11.99,9.49,7.219389,4.09
412697,4.09,4.4,7.54,1.3,4.0,7.0,4.49,10,20,4,...,2,6,15,105.0,1,1,11.99,9.49,7.162872,4.09
412698,4.09,4.4,7.54,1.4,4.0,7.0,4.49,10,20,4,...,2,6,20,100.0,1,1,11.99,9.49,7.042960,4.09
412699,4.09,5.0,7.54,1.3,4.0,7.0,4.49,10,20,4,...,2,6,25,95.0,1,1,11.99,9.49,6.762695,4.09
412700,4.09,5.0,7.74,10.0,4.0,7.0,4.49,10,20,4,...,2,6,30,90.0,1,1,11.99,9.49,6.663984,4.09


## Precision


In [127]:
# Calculate metrics
mse = mean_squared_error(y_test_bid, preds_bid)
mae = mean_absolute_error(y_test_bid, preds_bid)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")


Mean Squared Error: 783.0336
Mean Absolute Error: 5.7348


## Feature Importance

In [125]:
# Try to find out which features are most important
fi = pd.DataFrame(data=model_bid.feature_importances_, index=model_bid.feature_names_in_, columns=['importance'])

In [126]:
fi.sort_values(by='importance').plot(kind='barh', title='Feature Importance')
plt.show()

## Model ask

In [63]:
# Create model instance
model_ask = XGBRegressor(n_estimators=1000, early_stopping_rounds=50, max_depth=3, learning_rate=0.1)
# Fit model
model_ask.fit(x_train_ask, y_train_ask_imputed, eval_set=[(x_train_ask, y_train_ask_imputed), (x_test_ask, y_test_ask_imputed)], verbose=True)
# Make predictions  
preds_ask = model_bid.predict(x_test_ask)

[0]	validation_0-rmse:43.87469	validation_1-rmse:38.44890
[1]	validation_0-rmse:43.00069	validation_1-rmse:37.38821
[2]	validation_0-rmse:42.25999	validation_1-rmse:36.47016
[3]	validation_0-rmse:41.64486	validation_1-rmse:35.71929
[4]	validation_0-rmse:41.03766	validation_1-rmse:35.03506
[5]	validation_0-rmse:40.59047	validation_1-rmse:34.47045
[6]	validation_0-rmse:39.75267	validation_1-rmse:33.99353
[7]	validation_0-rmse:39.04881	validation_1-rmse:33.61120
[8]	validation_0-rmse:38.75067	validation_1-rmse:33.30324
[9]	validation_0-rmse:38.49233	validation_1-rmse:33.01062
[10]	validation_0-rmse:37.97544	validation_1-rmse:32.79739
[11]	validation_0-rmse:37.80023	validation_1-rmse:32.59361
[12]	validation_0-rmse:37.65645	validation_1-rmse:32.43683
[13]	validation_0-rmse:37.53605	validation_1-rmse:32.31402
[14]	validation_0-rmse:37.43278	validation_1-rmse:32.19483
[15]	validation_0-rmse:37.09128	validation_1-rmse:32.09289
[16]	validation_0-rmse:37.01476	validation_1-rmse:32.01803
[17]	va

In [64]:
results_ask = x_test_ask.copy()
results_ask['preds'] = preds_ask
results_ask['actual'] = y_test_ask_imputed

In [65]:
results_ask

,orderPrice_bid_best,orderQty_bid_best,orderPrice_ask_best,orderQty_ask_best,DA_price,dlvryMonth,dlvryDay,dlvryHour,dlvryWeekday,timeMonth,timeDay,timeHour,timeWeekday,timeMinute,exists_bid,exists_ask,preds,actual
8600869,0.01,5.0,9.00,5.0,6.64,10,20,6,6,10,19,15,5,24,1,1,0.823294,8.8
8600898,0.01,5.0,9.00,5.0,6.64,10,20,6,6,10,19,15,5,25,1,1,0.827207,8.8
8600927,0.01,5.0,9.00,5.0,6.64,10,20,6,6,10,19,15,5,26,1,1,0.827207,8.8
8600956,0.01,5.0,9.00,5.0,6.64,10,20,6,6,10,19,15,5,27,1,1,0.827207,8.8
8600985,0.01,5.0,9.00,5.0,6.64,10,20,6,6,10,19,15,5,28,1,1,0.825777,8.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10765075,12.62,4.3,16.85,1.5,12.85,12,31,22,1,12,31,20,1,55,1,1,13.864390,12.0
10765100,12.62,4.3,16.85,1.5,12.85,12,31,22,1,12,31,20,1,56,1,1,13.864390,12.0
10765125,12.62,4.3,16.85,1.5,12.85,12,31,22,1,12,31,20,1,57,1,1,13.864390,12.0
10765150,11.01,0.4,12.00,10.7,12.85,12,31,22,1,12,31,20,1,58,1,1,12.475830,12.0


## Precision

In [66]:
# Calculate metrics
mse = mean_squared_error(y_test_bid_imputed, preds_bid)
mae = mean_absolute_error(y_test_bid_imputed, preds_bid)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")

Mean Squared Error: 1905.0918
Mean Absolute Error: 10.2230


## Feature Importance

In [67]:
# Try to find out which features are most important
fi = pd.DataFrame(data=model_bid.feature_importances_, index=model_bid.feature_names_in_, columns=['importance'])

In [68]:
fi.sort_values(by='importance').plot(kind='barh', title='Feature Importance')
plt.show()